<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### Notes to use SAR properly:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

# 0 Global Settings and Imports

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import logging
import time

import numpy as np
import pandas as pd
import papermill as pm

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Feb 11 2019, 15:03:47) [MSC v.1915 64 bit (AMD64)]
Pandas version: 0.24.1


# 1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>,<Time>,[<Event Type>], [<Event Weight>]`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. Each event type can be assigned a different weight, for example, we might assign a “buy” event a weight of 10, while a “view” event might only have a weight of 1.

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [2]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 1.1 Download and use the MovieLens Dataset

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data['rating'] = data['rating'].astype(np.float32)

data.head()

,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### 1.2 Split the data using the python random splitter provided in utilities:

We utilize the provided `python_random_split` function to split into `train` and `test` datasets randomly at a 80/20 ratio.

In [4]:
train, test = python_random_split(data)

# 2 Train the SAR Model

### 2.1 Instantiate the SAR algorithm and set the index

In order to use SAR, we need to hash users and items and set an index in order for SAR matrix operations to work. First we instantiate SAR using the reference implementation provided in `SARSingleNode`:

In [5]:
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True
)

### 2.2 Train the SAR model on our training data, and get the top-k recommendations for our testing data

SAR first computes an item-to-item ***co-occurence matrix***. Co-occurence represents the number of times two items appear together for any given user. Once we have the co-occurence matrix, we compute an ***item similarity matrix*** by rescaling the cooccurences by a given metric (Jaccard similarity in this example). 

We also compute an ***affinity matrix*** to capture the strength of the relationship between each user and each item. Affinity is driven by different types (like *rating* or *viewing* a movie), and by the time of the event. 

Recommendations are achieved by multiplying the affinity matrix $A$ and the similarity matrix $S$. The result is a ***recommendation score matrix*** $R$. We compute the ***top-k*** results for each user in the `recommend_k_items` function seen below.

A full walkthrough of the SAR algorithm can be found [here](../02_model/sar_deep_dive.ipynb).

In [6]:
start_time = time.time()

model.fit(train)

train_time = time.time() - start_time
print("Took {} seconds for training.".format(train_time))

2019-03-07 11:33:34,440 INFO     Collecting user affinity matrix
2019-03-07 11:33:34,451 INFO     Calculating time-decayed affinities
2019-03-07 11:33:34,726 INFO     Creating index columns
2019-03-07 11:33:34,762 INFO     Building user affinity sparse matrix
2019-03-07 11:33:34,773 INFO     Calculating item co-occurrence
2019-03-07 11:33:35,048 INFO     Calculating item similarity
2019-03-07 11:33:35,054 INFO     Calculating jaccard
2019-03-07 11:33:35,275 INFO     Done training


Took 0.8455877304077148 seconds for training.


In [7]:
start_time = time.time()

top_k = model.recommend_k_items(test, remove_seen=True)

test_time = time.time() - start_time
print("Took {} seconds for prediction.".format(test_time))

2019-03-07 11:33:35,294 INFO     Calculating recommendation scores
2019-03-07 11:33:35,481 INFO     Removing seen items
2019-03-07 11:33:35,502 INFO     Getting top K


Took 0.28716468811035156 seconds for prediction.


In [8]:
display(top_k.head())

,userID,itemID,prediction
0,877,153,2.311135
1,877,234,2.311886
2,877,238,2.315627
3,877,168,2.393810
4,877,97,2.410769


### 5. Evaluate how well SAR performs 

We evaluate how well SAR performs with for a few common ranking metrics provided in the `PythonRankingEvaluation` classs in utilities. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with SAR.

In [9]:
eval_map = map_at_k(test, top_k, k=TOP_K)

In [10]:
eval_ndcg = ndcg_at_k(test, top_k, k=TOP_K)

In [11]:
eval_precision = precision_at_k(test, top_k, k=TOP_K)

In [12]:
eval_recall = recall_at_k(test, top_k, k=TOP_K)

In [13]:
print("Model:\t" + model.model_str,
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Model:	sar_ref
Top K:	10
MAP:	0.104852
NDCG:	0.370487
Precision@K:	0.321125
Recall@K:	0.174170


In [14]:
# Record results with papermill for tests
pm.record("map", eval_map)
pm.record("ndcg", eval_ndcg)
pm.record("precision", eval_precision)
pm.record("recall", eval_recall)
pm.record("train_time", train_time)
pm.record("test_time", test_time)